<a href="https://colab.research.google.com/github/khodizoda/42_ft_printf/blob/master/SiteWorkerIntegratedV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## **Set up and update**


In [ ]:
# from google.colab import auth
# credentials = auth.authenticate_user()

!apt-get update && apt-get upgrade
!apt install chromium-chromedriver
!pip install centaurMiner==0.0.8
!pip install import_ipynb
!pip install tld

!apt autoremove

!curl 'https://raw.githubusercontent.com/aivscovid19/data_pipeline/gulnoza/JobDispatcher.ipynb' > JobDispatcher.ipynb
!curl 'https://raw.githubusercontent.com/khodizoda/ai_vs_covid19/master/IbmcRuMiner.ipynb' > IbmcRuMiner.ipynb

In [20]:
import import_ipynb
import random, time
import pandas as pd
from pandas.io import gbq
from tld import get_fld
from JobDispatcher import JobDispatcher
from IbmcRuMiner import IbmcRuMiner

class SiteWorkerIntegrated():
  '''
  SiteWorkerIntegrated class uses `site_worker_factory` method
  to send data mining request to a domain-specific SiteWorker
  given urls dataframe.

  Attributes:
    max_threshold (int): A default value of how many articles to upload
                          to a BigQuery table at a time.
    min_delay (int): A default value of min seconds to wait before the
                      next request to a website is sent.
    max_delay (int): A default value of max seconds to wait before the
                      next request to a website is sent.
  '''

  def __init__(self):
    self.max_threshold = 50
    self.min_delay = 0.1
    self.max_delay = 2

  def send_request(self, urls_df, limit):
    ''' Finds domain from urls dataframe and sends request
    to a domain-specific SiteWorker using site_worker_factory class method.
    '''
    try:
      url = urls_df.at[0, 'article_url'] 
      domain = get_fld(url)
      print(domain)

      self.site_worker_factory(domain, urls_df, limit, self.driver_path)
  
    except:
      print('The urls dataframe is empty')

  def scrape_data(self, miner, urls_df, article_schema, limit=100):
    ''' Scrapes data given urls' dataframe, and limit of articles to scrape.
    Uploads data to a given BigQuery table and calls `update_job_status` method
    from `JobDispatcher` class from `JobDispatcher` module
    to update `status` of the job to `done`.

    Attributes:
      urls_df (pandas dataframe): A urls dataframe, to get a list of
                                    article urls to scrape.
      limit (int): A limit of articles to scrape. Default is 100.
    '''

    urls = [url for url in list(urls_df['article_url'])]
    data = []
    prev_count = 0
    for count, url in enumerate(urls, 1):
      miner.gather(url)
      data.append(miner.results)
      if (count == self.max_threshold or count == limit or count == len(urls)):
        articles_list = list(filter(lambda i:
                            i['title'] != None and len(i['title']) != 0 and
                            i['abstract'] != None and len(i['abstract']) != 0,
                            data[prev_count : count])) 
        articles_df = pd.DataFrame(articles_list)
        articles_df.to_gbq(destination_table=f'{self.article_table}',
                  project_id=self.project_id,
                  if_exists='append',
                  table_schema=article_schema,
                  credentials=self.credentials)
        JobDispatcher(self.credentials,
                      self.project_id,
                      self.url_table).update_job_status(urls_df.iloc[prev_count : count].copy())
        prev_count = count
      time.sleep(self.min_delay + self.max_delay * random.random())
    return articles_df
  
  @classmethod
  def init(cls, credentials, project_id, url_table, article_table, driver_path=None):
    ''' Initializes `SiteWorkerIngrated` class

    Attributes:
      credentials (str): Credentials, either from user_account or service_account,
                          to authenticate to Google Cloud APIs.
      project_id (str): A project_id on Google Cloud Platform.
      url_table (str): A url_table to use to retrieve urls_dataframe from,
                        in form of `dataset_id.table_id`.
      article_table (str): An article_table to use to upload scraped data to,
                        in form of `dataset_id.table_id`.
      driver_path (str): A driver path to a chromium-chromedriver.
    '''
    cls.credentials = credentials
    cls.project_id = project_id
    cls.url_table = url_table
    cls.article_table = article_table
    cls.driver_path = driver_path

  @classmethod
  def site_worker_factory(cls, domain_name, urls_df, limit=100, driver_path=None):
    ''' Sends a scraping request to a domain-specific SiteWorker

    Attributes:
      domain_name (str): A domain.
      urls_df (pandas dataframe): A urls dataframe.
      limit (int): A limit of articles to scrape. Default is 100.
      driver_path (str): A driver path to a chromium-chromedriver.
    '''
    site_worker = {'ibmc.msk.ru': IbmcRuSiteWorker(urls_df, limit, driver_path).scrape_articles()
    
    }

In [21]:
class IbmcRuSiteWorker(SiteWorkerIntegrated):
  '''
  SiteWorker for http://pbmc.ibmc.msk.ru/

  Attributes:
    urls_df (pandas dataframe): A urls dataframe.
    limit (int): A limit of articles to scrape. Default is 100.
    driver_path (str): A driver path to a chromium-chromedriver.
    article_schema (list of dicts): An default article_schema for
                                      a given SiteWorker.
  '''
  def __init__(self, urls_df, limit=100, driver_path=None):
    super().__init__()
    self.urls_df = urls_df
    self.limit = limit
    self.driver_path = driver_path
    self.article_schema = [
      {'name': 'abstract',                'type': 'STRING', 'mode': 'REQUIRED'},
      {'name': 'authors',                 'type': 'STRING'                    },
      {'name': 'date_publication',        'type': 'DATE'                      },
      {'name': 'doi',                     'type': 'STRING'                    },
      {'name': 'extra_link',              'type': 'STRING'                    },
      {'name': 'keywords',                'type': 'STRING'                    },
      {'name': 'license',                 'type': 'STRING'                    },
      {'name': 'organization_affiliated', 'type': 'STRING'                    },
      {'name': 'pubmed_link',             'type': 'STRING'                    },
      {'name': 'source',                  'type': 'STRING'                    },
      {'name': 'title',                   'type': 'STRING', 'mode': 'REQUIRED'},
      {'name': 'translated_link',         'type': 'STRING',                   },
      {'name': 'url',                     'type': 'STRING', 'mode': 'REQUIRED'},
      {'name': 'date_aquisition',         'type': 'DATE'                      },           
    ]

  def scrape_articles(self):
    miner = IbmcRuMiner.IbmcEngine(IbmcRuMiner.IbmcLocations, driver_path=self.driver_path)
    self.scrape_data(miner, self.urls_df, self.article_schema, self.limit)